# QDA Planning

In [137]:
import numpy as np
import qda

Goal: Fit a Gaussian model for each class and find misclassification rate on test data.

Need:
- Functions to estimate $\hat{\pi}_k$, $\hat{\mu}_k$, and $\hat{\Sigma}_k$ for each class $k$
- Functions to compute quadratic descriminant functions $\delta_k(x)$ for a given $x$ (or a whole dataset $\mathbf{X})$)
- Function to classify an $x$ or a dataset $\mathbf{X}$ using then $\delta_k(x)$
- Function to calculate misclassification rate given estimates $\hat{y}$ and real values $y$
- Function to convert classes to integers 0,..,K-1 and record in dictionary
- Function to convert $y$ to response matrix

In [124]:
def gen_class_codes(y):
    '''
    Code distinct entries of y as 0,..,K-1. Ordered with floats/ints
    first followed by strings.
    
    Parameters
    ----------
    y: Numpy array of size (N, ) consisting of K distinct class labels
       Entries must be floats of strings
       
    Returns
    -------
    code_to_class: dict with keys 0,..,K-1 and values class labels
    
    class_to_code: dict with class labels as keys and values 0,..,K-1
                   Inverse of code_to_class
    '''
    # Get ordered class lables
    sorted_values = np.sort(np.unique(y))
    
    # Generate dictionaries
    code_to_class = dict(enumerate(sorted_values))
    class_to_code = {v:k for k, v in code_to_class.items()}
    
    return code_to_class, class_to_code

In [126]:
def encode_classes(y, class_to_code):
    '''
    Replace class labels with codes
    '''
    return np.array([class_to_code[i] for i in y])

def decode_classes(y, code_to_class):
    '''
    Replace codes with class labels
    '''
    return np.array([code_to_class[i] for i in y])

In [73]:
def gen_indicator_responses(y):
    '''
    Turn output vector with entries 0,..,K-1 into
    indicator reponse matrix
    
    Parameters
    ----------
    y: Numpy array of shape (N,) with entries 0,..,K-1
       
    Returns
    -------
    Y: Numpy array of shape (N, K)
       1s in position [i, y[i]], 0s elsewhere
    '''
    N = y.shape[0]
    
    Y = np.zeros(shape=(N, K))
    Y[range(N), y] = 1
    
    return Y

In [3]:
def est_params(X, Y):
    '''
    Estimate prior probabilities, class means, and class variances
    for quadratic discriminant analysis.
    
    Parameters
    ----------
    X: Numpy array of shape (N, p) containing input data
    
    Y: Numpy array of shape (N, K), indicator response matrix
    
    Returns
    -------
    prior_prob: Numpy array of shape (K, )
                Estimates of prior probabilities of each class
                
    sample_means: Numpy array of shape (K, p)
                  kth row is sample mean of class k
    
    sample_vars: Numpy array of shape (K, p, p)
                 kth entry is sample covariance matrix of class k
    '''
    N, p = X.shape
    _, K = Y.shape
    
    prior_prob = np.sum(Y, axis=0) / np.sum(Y)
    
    sample_means = np.empty(shape=(K, p))
    sample_vars = np.empty(shape=(K, p, p))
    
    for k in range(K):
        class_inputs = X[Y[:, k] == 1, :]
        
        sample_means[k] = np.mean(class_inputs, axis = 0)
        
        sample_vars[k] = (class_inputs - sample_means[k]).T @ (class_inputs - sample_means[k])
        sample_vars[k] /= class_inputs.shape[0] - K
        
    return prior_prob, sample_means, sample_vars

In [52]:
def discriminant_func(X, prior_prob, sample_means, sample_vars):
    '''
    Apply all quadratic discriminant functions to input data X.
    In practice last 3 parameters are estimated using est_params
    
    Parameters
    ----------
    X: Numpy array of shape (N, p) consisting of input data
    
    prior_prob: Numpy array of shape (K, )
                Prior probabilities for each class
    
    class_means: Numpy array of shape (K, p)
                 kth row is population mean of class k
    
    class_vars: Numpy array of shape (K, p, p)
                kth entry is covariance matrix of class k
    
    Returns
    -------
    delta: Numpy array of shape (N, K)
           ith row gives K discriminant funcs applied to ith data point
    
    '''
    # Get eigendecomposition of each covariance matrix
    eval, evect = np.linalg.eigh(class_vars)
    
    # Calculate D^(-1/2) for each set of evalues
    diag = np.apply_along_axis(np.diag, axis=1, arr=np.sqrt(1/eval))
    
    # Initialise K copies of data to apply K disc func
    delta = np.array([X] * K)
    
    # Calculate Mahalanobis distance for each class
    delta = delta - class_means[:, np.newaxis, :]    
    delta = delta @ evect @ diag    
    delta = np.sum(delta**2, axis=2).T
    
    # Add constant terms
    delta = prior_prob - delta / 2 - np.sum(np.log(eval), axis=1) / 2

    return delta

In [53]:
def classify(X, prior_prob, sample_means, sample_vars):
    '''
    Classify each data point in X
    In practice last 3 parameters are estimated using est_params
    
    Parameters
    ----------
    X: Numpy array of shape (N, p) consisting of input data
    
    prior_prob: Numpy array of shape (K, )
                Prior probabilities for each class
    
    class_means: Numpy array of shape (K, p)
                 kth row is population mean of class k
    
    class_vars: Numpy array of shape (K, p, p)
                kth entry is covariance matrix of class k
    
    Returns
    -------
    y_est: Numpy array of shape (N, ) with entries in range(K)
           ith entry gives estimated class for ith input
    '''
    # Apply discriminant functions
    delta = discriminant_func(X)
    
    y_est = np.argmax(delta, axis=1)
    
    return y_est

In [87]:
def classification_error(y, y_est):
    '''
    Returns classification error in estimating y by y_est
    
    Parameters
    ----------
    y: Numpy array of shape (N,) with entries 0,..,K-1
       True class outputs
       
    y_est: Numpy array of shape (N,) with entries 0,..,K-1
           Estimated class outputs
    
    Returns
    -------
    err: Float between 0 and 1
         Number of misclassified data points over total number
    '''
    N = y.shape[0]
    err = np.sum((y != y_est).astype(int)) / N
    
    return err

I should test this by generating some multivariate Gaussian data and using this to estimate the parameters

In [80]:
y = np.array([
    0, 0, 0, 1, 1, 1
])

Y = np.array([
    [1, 0],
    [1, 0],
    [1, 0],
    [0, 1],
    [0, 1],
    [0, 1]
])

X = np.array([
    [3, 0],
    [2, 1],
    [1, 1],
    [1, 1],
    [1, 2],
    [0, 3]
])

N, p = X.shape
_, K = Y.shape

prior_prob, sample_means, sample_vars = est_params(X, Y)

Next: Function for quadratic discriminant functions

In [165]:
k = 0
sample_mean = sample_means[k]
sample_var = sample_vars[k]

In [166]:
eval, evect = np.linalg.eigh(sample_vars[0])

In [173]:
eval, evect = np.linalg.eigh(sample_var)

delta = (X - sample_mean) @ evect @ np.diag(np.sqrt(1/eval))
delta = np.sum(delta**2, axis=1)
delta

Now do this for all $k$ simultaneously

In [5]:
sample_means

array([[2.        , 0.66666667],
       [0.66666667, 2.        ]])

In [215]:
eval, evect = np.linalg.eigh(sample_vars)

In [10]:
np.array([X] * K)

array([[[3, 0],
        [2, 1],
        [1, 1],
        [1, 1],
        [1, 2],
        [0, 3]],

       [[3, 0],
        [2, 1],
        [1, 1],
        [1, 1],
        [1, 2],
        [0, 3]]])

In [15]:
sample_vars

array([[[ 2.        , -1.        ],
        [-1.        ,  0.66666667]],

       [[ 0.66666667, -1.        ],
        [-1.        ,  2.        ]]])

In [18]:
evect

array([[[-0.47185793, -0.8816746 ],
        [-0.8816746 ,  0.47185793]],

       [[-0.8816746 , -0.47185793],
        [-0.47185793,  0.8816746 ]]])

In [25]:
np.apply_along_axis(np.diag, axis=1, arr=np.array([[0, 1], [1, 2]]))

array([[[0, 0],
        [0, 1]],

       [[1, 0],
        [0, 2]]])

In [21]:
np.apply_along_axis(np.diag, axis=1, arr=eval)

array([[[0.13148291, 0.        ],
        [0.        , 2.53518376]],

       [[0.13148291, 0.        ],
        [0.        , 2.53518376]]])

In [17]:
evect @ np.apply_along_axis(np.diag, axis=1, arr=eval) @ evect.T

array([[[ 2.        ,  1.10940039],
        [-1.        , -0.46225016]],

       [[-0.46225016, -1.        ],
        [ 1.10940039,  2.        ]]])

In [54]:
# Apply QD functions to each row of X
eval, evect = np.linalg.eigh(sample_vars)
diag = np.apply_along_axis(np.diag, axis=1, arr=np.sqrt(1/eval))

delta = np.array([X] * K)
delta = delta - sample_means[:, np.newaxis, :]
delta = delta @ evect @ diag
delta = np.sum(delta**2, axis=2)
delta = delta.T# kth row is delta_k(X)

delta = prior_prob - delta / 2 - np.sum(np.log(eval), axis=1) / 2

# Classify data points
y_est = np.argmax(delta, axis=1)

In [58]:
Y_est = np.zeros(shape=(len(y_est), K))
Y_est[range(len(y_est)), y_est] = 1

In [66]:
Y_est[range(len(y_est)), y_est] = 1

In [76]:
Y_est = gen_indicator_responses(y_est)

In [78]:
Y - Y_est

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 1., -1.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]])

In [86]:
np.sum((y != y_est).astype(int))

1

In [88]:
classification_error(y, y_est)

0.16666666666666666